# L'API utilisée : **OpenSky Network** 
c'est un projet collaboratif qui fournit des données de traffic aérien en temps réel. Elle permet d’accéder aux informations ADS-B (Automatic Dependent Surveillance–Broadcast) transmises par les avions, captées par un réseau mondial de récepteurs.

L'url qu'on utilise est "https://opensky-network.org/api/states/all" et elle retourne un json de la forme : 

{
  "time": 1711291270,
  "states": [
    [
      "a00001",        # 0: icao24 (identifiant unique de l'avion)
      "AFR123",        # 1: callsign (indicatif du vol, ex: "AFR123" = Air France 123)
      "France",        # 2: origin_country
      1711291267,      # 3: time_position
      1711291268,      # 4: last_contact
      2.3522,          # 5: longitude
      48.8566,         # 6: latitude
      11000.0,         # 7: baro_altitude (en mètres)
      False,           # 8: on_ground (si l'avion est au sol)
      250.0,           # 9: velocity (vitesse horizontale en m/s)
      ...
    ]
  ]
}



## Imports et récupération des données

In [ ]:
import requests
from kafka import KafkaProducer
import json
import time

# Fonction pour récupérer les données brutes depuis l'API OpenSky
def get_opensky_data():
    url = "https://opensky-network.org/api/states/all"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data.get("states", [])
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return []

## Définition des envois

### Envoi pour le comptage par région

In [ ]:
# Fonction pour compter le nombre d'avions par pays d'origine
def process_region_counts(states):
    region_counts = {}
    for state in states:
        icao24, callsign, origin_country = state[:3]
        region_counts[origin_country] = region_counts.get(origin_country, 0) + 1
    return region_counts

# Envoi des comptages par région (format clé/valeur)
def send_region_counts(topic, region_counts):
    producer = KafkaProducer(
        bootstrap_servers='kafka1:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    for key, value in region_counts.items():
        producer.send(topic, value={"region": key, "count": value})
    producer.flush()
    print(f"Sent region counts to topic '{topic}'")

# Boucle principale, envoi des données toutes les 10 secondes    
def main():
    topic_regions = "opensky-planes"

    while True:
        states = get_opensky_data()

        # Envoi du comptage par région
        region_data = process_region_counts(states)
        send_region_counts(topic_regions, region_data)

        time.sleep(10)

if __name__ == "__main__":
    main()


### Envoi pour les inormations sur les avions avec un callsign non vide

In [ ]:
# Fonction pour filtrer les avions ayant un callsign non vide
def process_callsign_data(states):
    filtered = []
    for state in states:
        callsign = state[1]
        if callsign and callsign.strip() != "":
            filtered.append({
                "callsign": callsign.strip(),
                "latitude": state[6],
                "longitude": state[5],
                "baro_altitude": state[7],
                "velocity": state[9]
            })
    return filtered

# Envoi d'une liste de messages (avions avec callsign) vers un topic Kafka
def send_to_kafka(topic, data_list):
    producer = KafkaProducer(
        bootstrap_servers='kafka1:9092',
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    for item in data_list:
        producer.send(topic, value=item)
    producer.flush()
    print(f"Sent {len(data_list)} records to topic '{topic}'")


# Boucle principale, envoi des données toutes les 10 secondes
def main():
    topic_callsigns = "opensky-callsigns"

    while True:
        states = get_opensky_data()

        # Envoi des avions avec callsign
        callsign_data = process_callsign_data(states)
        send_to_kafka(topic_callsigns, callsign_data)

        time.sleep(10)

if __name__ == "__main__":
    main()
